#### PI-NN test for TOC data
#### alpha = 0.1

In [7]:
import numpy as np

from models.tensorflow_pi import TF_PI
from utils.data_preprocess import pre_process
from utils.plot_func import plot_pi_toc
from utils.tools import Loss_func

TOC_file = './data/well_3/TOC_data_liushagang.csv'
# TOC_file = './data/well_3/TOC_data.csv'
welllog_file = './data/well_3/welllog_data.csv'

stratum_depth = [2402.4, 2543.3, 2790.3, 2995]
stratum_name = ['Liushagang_1', 'Liushagang_2', 'Liushagang_3']
logging_data, toc_data, unit, merge_toc, _ = pre_process(
    TOC_file, welllog_file, stratum_depth)
X = merge_toc[merge_toc.columns.difference(['DEPT', 'TOC'])].to_numpy()
y = merge_toc['TOC'].to_numpy()

test_data = logging_data[logging_data.columns.difference(['DEPT'])]

alpha = 0.15
tf_pi = TF_PI(alpha, weight=[1, 0.8, 0.5])
history = tf_pi.fit(X, y, epochs=800)
result = tf_pi.predict(test_data)
model_loss = Loss_func(result, y, (1-alpha),
                      merge_toc['DEPT'].to_numpy(), logging_data['DEPT'].to_numpy())

print("PICP = {}, PIMW = {}, PIAD = {}, PIEI = {}".format(model_loss.picp, model_loss.pimw,
                                                          model_loss.piad, model_loss.loss))

plot_pi_toc(result, merge_toc, logging_data, stratum_depth,
            stratum_name, model_loss.outlier, model_std=None)

100%|██████████| 800/800 [00:12<00:00, 62.23it/s]


PICP = 0.87, PIMW = 0.13, PIAD = 0.11, PIEI = 0.31


***
#### Different alphas test
#### alphas = [0.2, 0.15, 0.1, 0.05]

In [4]:
from models.ensemble_pi import diff_alphas_PI
from utils.data_preprocess import pre_process
from utils.plot_func import subplot_fit_process, subplot_multi_toc

TOC_file = './data/well_3/TOC_data_liushagang.csv'
welllog_file = './data/well_3/welllog_data.csv'

stratum_depth = [2402.4, 2543.3, 2790.3, 2995]
stratum_name = ['Liushagang_1', 'Liushagang_2', 'Liushagang_3']
logging_data, toc_data, unit, merge_toc, _ = pre_process(
    TOC_file, welllog_file, stratum_depth)
X = merge_toc[merge_toc.columns.difference(['DEPT', 'TOC'])].to_numpy()
y = merge_toc['TOC'].to_numpy()
test_data = logging_data[logging_data.columns.difference(['DEPT'])]
alphas = [0.2, 0.15, 0.1, 0.05]
result_all, index_all, outlier_list, hist = diff_alphas_PI(
    X, y, test_data, merge_toc['DEPT'].to_numpy(), logging_data['DEPT'].to_numpy(), 
    alphas, weight=[1, 0.7, 0.5])

print(index_all)
subplot_multi_toc(result_all, alphas, outlier_list,
                       merge_toc['DEPT'].to_numpy(), logging_data['DEPT'].to_numpy(), y, stratum_depth, stratum_name)

subplot_fit_process(hist, alphas)

100%|██████████| 1000/1000 [00:23<00:00, 43.29it/s]


184/184 [==============================] - 0s 929us/step


100%|██████████| 1000/1000 [00:22<00:00, 43.81it/s]


184/184 [==============================] - 0s 929us/step


100%|██████████| 1000/1000 [00:23<00:00, 43.09it/s]


...

100%|██████████| 1000/1000 [00:23<00:00, 42.77it/s]

  1/184 [..............................] - ETA: 11s

184/184 [==============================] - 0s 1ms/step


100%|██████████| 4/4 [01:33<00:00, 23.47s/it]

[array([0.78, 0.1 , 0.07, 0.23]), array([0.87, 0.18, 0.13, 0.38]), array([0.91, 0.3 , 0.22, 0.55]), array([0.91, 0.13, 0.11, 1.01])]


***
##### Ensemble PI-NN test
##### alpha = 0.1

In [3]:
import numpy as np

from models.ensemble_pi import Bootstrap_PI
from utils.data_preprocess import pre_process
from utils.plot_func import plot_pi_toc, plot_simple_boundary
from utils.tools import Loss_func

TOC_file = './data/well_3/TOC_data_liushagang.csv'
welllog_file = './data/well_3/welllog_data.csv'

stratum_depth = [2402.4, 2543.3, 2790.3, 2995]
stratum_name = ['Liushagang_1', 'Liushagang_2', 'Liushagang_3']
logging_data, toc_data, unit, merge_toc, _ = pre_process(
    TOC_file, welllog_file, stratum_depth)
X = merge_toc[merge_toc.columns.difference(['DEPT', 'TOC'])].to_numpy()
y = merge_toc['TOC'].to_numpy()
test_data = logging_data[logging_data.columns.difference(['DEPT'])]


alpha = 0.1
model = Bootstrap_PI(5, alpha, [1, 0.3, 0.5], bootstrap_method='prop_of_data')
result_all, hist, y_pred_gauss_mid, y_pred_gauss_dev, up_low = model.fit_predict(
    X, y, test_data)

model_loss = Loss_func(up_low, y, (1-alpha),
                     toc_data['DEPT'].to_numpy(), logging_data['DEPT'].to_numpy())
print("PICP = {}, PIMW = {}, PIAD = {}, Loss = {}".format(model_loss.picp, model_loss.pimw,
                                                          model_loss.piad, model_loss.loss))
plot_pi_toc(up_low, merge_toc, logging_data, stratum_depth,
            stratum_name, model_loss.outlier, y_pred_gauss_dev)

100%|██████████| 1000/1000 [00:24<00:00, 41.42it/s]


184/184 [==============================] - 0s 1ms/step


100%|██████████| 1000/1000 [00:23<00:00, 42.31it/s]

 55/184 [=======>......................] - ETA: 0s

184/184 [==============================] - 0s 1ms/step


...

100%|██████████| 1000/1000 [00:21<00:00, 45.63it/s]

  1/184 [..............................] - ETA: 8s

184/184 [==============================] - 0s 891us/step


100%|██████████| 5/5 [01:57<00:00, 23.45s/it]

PICP = 0.91, PIMW = 0.34, PIAD = 0.12, Loss = 0.49


***
##### GPR TOC Test
##### alpha = 0.1

In [1]:
import numpy as np
from scipy import stats
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from sklearn.gaussian_process.kernels import ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel

from utils.data_preprocess import pre_process
from utils.plot_func import plot_pi_toc
from utils.tools import Loss_func

TOC_file = './data/well_3/TOC_data_liushagang.csv'
welllog_file = './data/well_3/welllog_data.csv'

stratum_depth = [2402.4, 2543.3, 2790.3, 2995]
stratum_name = ['Liushagang_1', 'Liushagang_2', 'Liushagang_3']
logging_data, toc_data, unit, merge_toc, _ = pre_process(
    TOC_file, welllog_file, stratum_depth)
X = merge_toc[merge_toc.columns.difference(['DEPT', 'TOC'])].to_numpy()
y = merge_toc['TOC'].to_numpy()
test_data = logging_data[logging_data.columns.difference(['DEPT'])]

alpha = 0.1

# kernel = C(1e-1, (1e-5, 1e5)) * RBF(1e-1, (1e-5, 1e5))
# gpr = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=100, alpha=0.1).fit(X,y)
kernel = DotProduct() + WhiteKernel()
gpr = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=100,random_state=10).fit(X,y)
result, std = gpr.predict(test_data, return_std=True)
up_low = stats.norm.interval(1-alpha, loc=result, scale=std)

up_low = np.array(up_low).T

model_loss = Loss_func(up_low, y, (1-alpha),
                     toc_data['DEPT'].to_numpy(), logging_data['DEPT'].to_numpy())
print("PICP = {}, PIMW = {}, PIAD = {}, Loss = {}".format(model_loss.picp, model_loss.pimw,
                                                          model_loss.piad, model_loss.loss))

plot_pi_toc(up_low, merge_toc, logging_data, stratum_depth,
            stratum_name, model_loss.outlier)

f:\Anaconda\envs\pi\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but GaussianProcessRegressor was fitted without feature names
  warnings.warn(


PICP = 0.96, PIMW = 0.8, PIAD = 0.2, Loss = 1.92
